In [1]:
# change the directory
import os 
os.chdir("c:\\Users\\jzou2\\Fall 2024-2025\\Project\\OptiMed")
from typing import TypedDict, List


# import relevant packages
from agents.base_llm import OpenAILLMs
from agents.master import MasterWorkflow
from agents.user_interaction.user_inputter import UserLLM

from agents.diagnosis.diagnosis_coordinator import DiagnosticCoordinator

from agents.diagnosis.generalist import GeneralistLLM

In [ ]:
# initial set up
class Report(TypedDict):
    """
    Defines the state for the multi-agent workflow.
    Includes messages, patient info, and workflow states.
    """
    patient_info: str
    messages: List[dict]
    diagnosis: str
    recommendations: str
    appointment_details: str


# Mock report with no messages
mock_diagnostic_report_no_messages: Report = {
    "patient_info": "Patient: John Doe, Age: 45, Gender: Male, Medical History: Hypertension, Asthma, Allergies: Penicillin. His Current Symptoms Is Shortness Of Breath And Chest Pain",
    "messages": [],
    "diagnosis": "",
    "recommendations": "",
    "appointment_details": ""
}

# Mock report with messages
mock_diagnostic_report_yes_messages: Report = {
    "patient_info": "Patient: John Doe, Age: 45, Gender: Male, Medical History: Hypertension, Asthma, Allergies: Penicillin. His Current Symptoms Is Shortness Of Breath And Chest Pain",
    "messages": [{"role": "generalist", "content": "This might be a heart attack, I would go to the recommender and suggest any diagnostic testing"}],  
    "diagnosis": "heart attack",
    "recommendations": "",
    "appointment_details": ""
}


#### Test LLMs and Agents

In [ ]:
def test_base_llm():
    test_agent = OpenAILLMs()
    print(test_agent.prompt)
    test_agent_result = test_agent("What is the weather today")
    return test_agent_result
print(test_base_llm())

In [ ]:
def test_master():
    master_agent = MasterWorkflow()
    output = master_agent.test_call()
    return output
print(test_master())

In [2]:
def test_user():
    "my symptoms are shortness of breath and acute chest pain"
    "32"
    "i do not have any medical history"
    user_agent = UserLLM()
    output = user_agent()
    return output
print(test_user())



> Entering new AgentExecutor chain...
Thought and Reasoning: I need to gather information about the user's symptoms, age, and medical history to provide a concise summary. Since no details have been provided yet, I will start by asking about the symptoms.
Action: get_user_input
Action Input: "Can you please describe your symptoms?"DONEThought and Reasoning: The user has indicated that they are done providing information, so I will not prompt for any further details. 
Final Answer: No summary can be provided as no information was gathered.

> Finished chain.
No summary can be provided as no information was gathered.


#### Test Synergies

In [2]:
def master_to_user():
    "my symptoms are shortness of breath and acute chest pain, age 32, and i have no medical history"
    master_agent = MasterWorkflow()
    output = master_agent.run()
    return output
print(master_to_user())



> Entering new AgentExecutor chain...
Thought and Reasoning: The first step in completing the form is to fill out the patient input. Therefore, the appropriate agent to delegate this task to is the USER INTERACTION agent. 
Action: USER INTERACTION
Final Answer: USER INTERACTION

> Finished chain.


> Entering new AgentExecutor chain...
Thought and Reasoning: I need to gather information about the user's symptoms, age, and medical history to provide a concise summary. Since no details have been provided yet, I will start by asking about the symptoms.  
Action: get_user_input  
Action Input: "Can you please describe your symptoms?"  "my symptoms are shortness of breath and acute chest pain, age 32, and i have no medical history"Thought and Reasoning: I have gathered information about the user's symptoms, age, and medical history. The user is 32 years old, experiencing shortness of breath and acute chest pain, and has no medical history. Since I have all the necessary details, I can now

KeyboardInterrupt: Interrupted by user

#### Main Workflow

In [ ]:
# creates class 
final_workflow = MasterWorkflow() # called once

# starts with initial form
initiate = final_workflow()